Proof of Concept - Manipulate datasets in the project
===

In [6]:
# Library Utilitaire pour la gestion de projet
import PythonLib

from os.path import splitext, basename

## CSV

In [7]:
import pandas as pd

author= "Mark S. Daskin"
topic = "NETWORK-AND-DISCRETE-LOCATION"

dataset="49-nodes.csv"

dataset_file = "../Datasets/" + author + "/" + dataset

data = pd.read_csv(dataset_file)

In [8]:
data.head()

,ID,LONGITUDE,LATITUDE,STATE POP,CITY POP,FIXED COST,CITY
0,1,121.467360,38.56685,29760021,369365,115800,< Sacramento CA >
1,2,73.799017,42.66575,17990455,101082,101800,< Albany NY >
2,3,97.750522,30.30588,16986510,465622,72600,< Austin TX >
3,4,84.281399,30.45700,12937926,124773,72400,< Tallahassee FL >
4,5,76.884503,40.27605,11881643,52376,38400,< Harrisburg PA >


## Mongodb

The data set may need some adjustment before behing send to the database, for instance the latitude needs to be corrected because by convention *west latitude* is represented by negative coordinates.

In [9]:
data["LONGITUDE"]= -data["LONGITUDE"]
data.head()

,ID,LONGITUDE,LATITUDE,STATE POP,CITY POP,FIXED COST,CITY
0,1,-121.467360,38.56685,29760021,369365,115800,< Sacramento CA >
1,2,-73.799017,42.66575,17990455,101082,101800,< Albany NY >
2,3,-97.750522,30.30588,16986510,465622,72600,< Austin TX >
3,4,-84.281399,30.45700,12937926,124773,72400,< Tallahassee FL >
4,5,-76.884503,40.27605,11881643,52376,38400,< Harrisburg PA >


### Create problem

In [10]:
# Generate a connection to our database
import pymongo
from pymongo import MongoClient

mongo_host='mongodb' #In a Docker compose environnement the host is the name of the container
mongo_port=27017

client = MongoClient(mongo_host, mongo_port)

# Collection where we wich to store our problem
problems = client[topic]["datasets"] # client.[db name].[collection name]

# Prepare the datas to be saved
data.index = data.index.map(str)
record = {
    "metadata": {
        "author": author,
        "topic": topic,
        "dataset": splitext(basename(dataset))[0]
    },
    "data": data.to_dict()
}

# Insert a new entry to the database
problem_id = problems.insert_one(record).inserted_id

DuplicateKeyError: E11000 duplicate key error collection: NETWORK-AND-DISCRETE-LOCATION.datasets index: metadata.author_-1_metadata.topic_-1_metadata.dataset_-1 dup key: { metadata.author: "Mark S. Daskin", metadata.topic: "NETWORK-AND-DISCRETE-LOCATION", metadata.dataset: "49-nodes" }

### Read problem

In [6]:
# fetch all problems, return an iterator
q = {} # query select the desired intances
f = {} # filter | select the desired fields
records = problems.find(q,f)

In [7]:
list(records)

[{'_id': ObjectId('5ef79259f3b7b4476d3be675')}]

## Create rules for the database

We want to preserve the integrity of our data, Therefore we need to add some rule to prevent incorrect read and write operations.

In [8]:
problems.create_index([('metadata.author', pymongo.DESCENDING),
                       ('metadata.topic', pymongo.DESCENDING),
                       ('metadata.dataset', pymongo.DESCENDING)], unique=True)

'metadata.author_-1_metadata.topic_-1_metadata.dataset_-1'